# Textanalyse historischer Daten mit NLTK, CLTK und spaCy

**Notebook für Videoaufnahmen zu Kapitel 10**

## 1. Setup

In dieser Zelle lädst du die benötigten Bibliotheken.

In [ ]:
import nltk
import matplotlib.pyplot as plt

try:
    import cltk
    from cltk import NLP as CLTK_NLP
except ImportError:
    cltk = None
    CLTK_NLP = None
    print("CLTK ist nicht installiert – CLTK-Beispiele werden übersprungen.")

try:
    import spacy
    nlp_en = spacy.load("en_core_web_sm")
except Exception as e:
    nlp_en = None
    print("spaCy oder das englische Modell sind nicht verfügbar:", e)

print("NLTK-Version:", nltk.__version__)

## 2. Einfache Tokenisierung mit NLTK

Beispiel mit einem Auszug aus der Magna Carta.

In [ ]:
from nltk.tokenize import word_tokenize

text_magna = (
    "In the first year of the reign of King John after the conquest of England, "
    "the barons of the realm demanded that the king confirm ancient liberties and laws."
)

print(text_magna)

tokens_split = text_magna.split()
print("\nNaive Tokenisierung mit split():")
print(tokens_split)
print("Anzahl Tokens (split):", len(tokens_split))

tokens_nltk = word_tokenize(text_magna)
print("\nTokenisierung mit NLTK:")
print(tokens_nltk)
print("Anzahl Tokens (NLTK):", len(tokens_nltk))

## 3. Tokenisierung und Lemmatisierung mit CLTK (Latein)

Beispiel mit Cicero (*In Catilinam* 1,1).

In [ ]:
latin_text = "Quo usque tandem abutere, Catilina, patientia nostra?"
print(latin_text)

if CLTK_NLP is None:
    print("\nCLTK ist nicht verfügbar. Überspringe dieses Beispiel.")
else:
    nlp_lat = CLTK_NLP("lat")
    doc_lat = nlp_lat(latin_text)

    print("\nTokens:")
    print(doc_lat.tokens)

    print("\nLemmata:")
    print(doc_lat.lemmata)

    print("\nPOS-Tags:")
    print(doc_lat.pos)

## 4. Stoppwörter und Wortfrequenzanalyse

Beispiel mit einem Auszug aus der Declaration of Independence.

In [ ]:
from nltk.probability import FreqDist
from nltk.corpus import stopwords

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

text_decl = (
    "When in the Course of human events it becomes necessary for one people to dissolve "
    "the political bands which have connected them with another, and to assume among the powers of the earth, "
    "the separate and equal station to which the Laws of Nature and of Nature's God entitle them, "
    "a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation."
)

tokens_decl = word_tokenize(text_decl)
fd_all = FreqDist(w.lower() for w in tokens_decl)
print("Häufigste Wörter (mit Stoppwörtern):")
print(fd_all.most_common(10))

en_stop = set(stopwords.words('english'))
tokens_decl_nostop = [w.lower() for w in tokens_decl if w.lower() not in en_stop]
fd_nostop = FreqDist(tokens_decl_nostop)
print("\nHäufigste Wörter (ohne Stoppwörter):")
print(fd_nostop.most_common(10))

plt.figure(figsize=(8,4))
fd_nostop.plot(15, cumulative=False)
plt.yscale('log')
plt.tight_layout()
plt.show()

## 5. NER mit spaCy

Beispieltext zur Französischen Revolution.

In [ ]:
text_revolution = (
    "In 1789 the French Revolution began in Paris, when members of the Third Estate "
    "formed the National Assembly and challenged the authority of King Louis XVI."
)
print(text_revolution)

if nlp_en is None:
    print("\nspaCy oder das englische Modell sind nicht verfügbar. Überspringe dieses Beispiel.")
else:
    doc = nlp_en(text_revolution)
    print("\nGefundene Entitäten:")
    for ent in doc.ents:
        print(ent.text, "->", ent.label_)

## 6. NER mit CLTK (Latein)

Beispiel mit einem Auszug aus der Vulgata (Lk 2,1–2).

In [ ]:
latin_ner_text = (
    "Factum est autem in diebus illis, exiit edictum a Caesare Augusto ut describeretur universus orbis. "
    "Hic descriptio prima facta est praeside Syriae Quirino."
)
print(latin_ner_text)

if cltk is None:
    print("\nCLTK ist nicht verfügbar. Überspringe NER-Beispiel.")
else:
    from cltk.tag import ner
    tagged = ner.tag_ner('lat', input_text=latin_ner_text, type=list)
    ents = [t for t in tagged if len(t) == 2]
    print("\nErkannte Entitäten:")
    print(ents)